# TAP snippets

This notebook will gather some code samples to query the JMMC tap services. Check profile at the begining to reach ou public or beta releases.

First samples relies on OiDB query.

Cheers,


# Get stats from OiDB using TAP 

In [1]:
import pyvo
from astropy.time import Time, TimeISOT

profile={"oidb":"","tap":""}
#uncomment next lines to get beta instances
#profile={"oidb":"-beta","tap":"-preprod"}

oidbMainUrl=f"http://oidb{profile['oidb']}.jmmc.fr"
oidbTapUrl=f"http://tap{profile['tap']}.jmmc.fr/vollt/tap"


oidbTap = pyvo.dal.TAPService(oidbTapUrl)

## Get last submission and last obs dates for every collections

In [2]:
query="SELECT obs_collection, MAX(subdate) AS last_subdate, MAX(t_max) AS last_tmax  FROM oidb GROUP BY obs_collection order by last_subdate"
table=oidbTap.search(query, maxrec=100, format="votable/td").to_table()
# convert mjd tmax to datetime
table["last_obsdate"]=Time(table["last_tmax"], format='mjd').to_datetime().astype('str')
table

obs_collection,last_subdate,last_tmax,last_obsdate
object,object,float64,str26
J/A+A/493/L17,2015-06-22T22:27:21Z,54281.35546875,2007-06-30 08:31:52.500000
J/A+A/536/A55,2015-06-22T22:35:46Z,55552.34375,2010-12-22 08:15:00
J/A+A/544/A91,2015-06-22T22:36:58Z,55886.5,2011-11-21 12:00:00
J/A+A/545/A130,2015-06-22T22:37:37Z,55136.1640625,2009-11-01 03:56:15
J/A+A/558/A149,2015-06-22T22:39:35Z,55791.359375,2011-08-18 08:37:30
J/A+A/558/A24,2015-06-22T22:40:58Z,56339.3984375,2013-02-16 09:33:45
J/A+A/559/A111,2015-06-22T22:41:49Z,56115.4375,2012-07-07 10:30:00
J/A+A/565/A71,2015-06-22T22:43:32Z,56196.39453125,2012-09-26 09:28:07.500000
6b3ce0c9-09e7-48a2-b117-f78d9a4d7458,2015-06-25T12:15:22Z,56784.16796875,2014-05-07 04:01:52.500000


### Extract information for a given obs_collection

In [3]:
obs_collection_id = "eso_vlti_import"

row = table[ table["obs_collection"] == obs_collection_id ][0]
last_subdate = row["last_subdate"]
last_obsdate= row["last_obsdate"]
print(f"{obs_collection_id} has last_subdate='{last_subdate}' and last_obsdate='{last_obsdate}'")

# Or specific query a single obs_collection
query=f"SELECT MAX(subdate) AS last_subdate, MAX(t_max) AS last_tmax  FROM oidb WHERE obs_collection='{obs_collection_id}'"
singletable=oidbTap.search(query, maxrec=100, format="votable/td").to_table()
singletable["last_obsdate"]=Time(singletable["last_tmax"], format='mjd').to_datetime().astype('str')
last_subdate = singletable["last_subdate"][0]
last_obsdate = singletable["last_obsdate"][0]
print(f"{obs_collection_id} has last_subdate='{last_subdate}' and last_obsdate='{last_obsdate}'")



eso_vlti_import has last_subdate='2025-01-10T06:00:05Z' and last_obsdate='2025-01-09 08:57:37.354239'
eso_vlti_import has last_subdate='2025-01-10T06:00:05Z' and last_obsdate='2025-01-09 08:57:37.354239'


## Get number of records by calib_level, obs_collection

In [4]:
query="SELECT count(*), calib_level, obs_collection FROM oidb GROUP BY obs_collection, calib_level ORDER BY calib_level"
table=oidbTap.search(query, maxrec=100, format="votable/td").to_table()

In [5]:
table

count_all,calib_level,obs_collection
int64,int32,object
37994,0,chara_import
486982,0,eso_vlti_import
6444,0,vegaobs_import
178,1,d259d56c-7304-4131-99d5-6c7304c13119
54,2,07f92587-f7ef-4f04-b925-87f7ef3f04c1
2,2,1d51fe4f-48b5-4de5-91fe-4f48b58de5df
2,2,3e71bedc-6d96-4675-b1be-dc6d9686752e
2,2,45840351-f65e-446c-8403-51f65e546c16
111,2,5866421b-f263-46c0-a642-1bf263a6c0bd


## Get number of records by quality_level,calib_level

In [6]:
query="SELECT count(*), quality_level, calib_level FROM oidb GROUP BY quality_level, calib_level"
table=oidbTap.search(query, maxrec=100, format="votable/td").to_table()
table

count_all,quality_level,calib_level
int64,int32,int32
178,0,1
680,0,2
258,0,3
1,1,2
3,3,2
2,3,3
23,4,2
143,4,3
3,5,2
